In [8]:
import pandas as pd
import numpy as np

from corpus_toolkit import corpus_tools as ct

In [54]:
df = pd.read_csv('Results/All_speeches_labelled.csv')

In [43]:
def perform_keyness(topic,year):
    sub_df = df.loc[(df.dynamic_label == topic) & (df.year_x == year)]

    dem_df = sub_df.loc[sub_df.party_y == 'D']
    rep_df = sub_df.loc[sub_df.party_y == 'R']
    
    dem_text = dem_df.speech_processed
    rep_text = rep_df.speech_processed
    
    d_freq = ct.frequency(ct.tokenize(dem_text))
    r_freq = ct.frequency(ct.tokenize(rep_text))
    
    d_keyness = ct.keyness(d_freq,r_freq,effect='log-ratio')
    r_keyness = ct.keyness(r_freq,d_freq,effect='log-ratio')
    return d_freq, d_keyness, r_freq, r_keyness

In [53]:
dfreq,dkey,rfreq,rkey = perform_keyness('abortion',2011)

AttributeError: 'dict' object has no attribute 'loc'

In [48]:
ct.head(df,hits=20)

be	2907
have	971
woman	936
abortion	523
bill	509
do	450
fund	353
service	319
health_care	310
health	276
republican	253
go	251
plan_parenthood	240
american	237
say	221
want	218
care	218
right	217
federal	212
make	206


In [47]:
ct.head(rf,hits=20)

be	2528
have	889
abortion	750
plan_parenthood	399
do	365
life	345
fund	329
bill	268
woman	247
say	242
american	210
time	194
taxpayer	188
use	186
go	176
child	176
protect	164
year	158
support	158
right	154


In [30]:
df.year_x

0         1983
1         1983
2         1983
3         1983
4         1983
          ... 
575603    2016
575604    2016
575605    2016
575606    2016
575607    2016
Name: year_x, Length: 575608, dtype: int64